# UAB - Aprendizaje Computacional  


## Cas Kaggle 

## Introducció

La base de dades consisteix en un conjunt de pel·lícules majoritàriament del segle 21 però també del segle 20.
Cada pel·lícula té marcat en quina plataforma de Streaming hi és.

Plataformes hi ha les 4 més importants avui dia: Netflix, Hulu, Prime Video i Disney +.
També conté la valoració de cadascuna de les pel·lícules a la web "Rotten Tomatoes". És una web molt coneguda i respectada a escala mundial sobre el món cinematogràfic.

La idea darrere d'aquest dataset és poder saber a quina plataforma anar quan es vulgui veure una pel·lícula en particular, i també fer un estudi sobre els conjunts de pel·lícules orientades a una franja d'edat i les plataformes que les ofereixen.

## Llibreries usades

In [2]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import svm, ensemble,metrics
from sklearn.model_selection import train_test_split, GridSearchCV,RandomizedSearchCV,KFold, StratifiedKFold, cross_val_score,LeaveOneOut
from numpy import mean,std
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score,precision_score, recall_score, f1_score, accuracy_score,roc_curve,classification_report,auc,precision_recall_curve
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
import warnings
import altair as alt
from imblearn.under_sampling import RandomUnderSampler
import timeit
from collections import Counter

ModuleNotFoundError: No module named 'altair'